In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.api import VAR 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic 

Load sensors data

In [2]:
data_sensor = pd.read_csv('../pruebas_datos/2020-12-03.csv')

In [3]:
data_sensor.head()

,time,measurement,site_id,mote_id,sensor_id,sensor_status,sensor_value
0,2020-12-03T00:00:00.263755,battery,parral,00-17-0d-00-00-59-d1-f1,FFC_SENSOR_BAT,FFC_SENSOR_OK,3.1876
1,2020-12-03T00:00:01.508517,battery,parral,00-17-0d-00-00-59-cd-ba,FFC_SENSOR_BAT,FFC_SENSOR_OK,3.1984
2,2020-12-03T00:00:02.233210,battery,parral,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_BAT,FFC_SENSOR_OK,3.1668
3,2020-12-03T00:00:03.477323,battery,parral,00-17-0d-00-00-59-a9-fb,FFC_SENSOR_BAT,FFC_SENSOR_OK,3.1248
4,2020-12-03T00:00:04.178922,battery,parral,00-17-0d-00-00-59-d1-e6,FFC_SENSOR_BAT,FFC_SENSOR_OK,3.1704


Load weather stations data

In [4]:
data_ws = pd.read_csv('../pruebas_datos/campanacura_parral-1.csv')

In [5]:
data_ws.head()

,Fecha Hora,Temp. promedio aire,Precipitación horaria,Humed. rel. promedio,Presión atmosférica,Radiación solar máx.,Veloc. máx. viento,Temp. Mínima,Temp. Máxima,Dirección del viento,Grados día (base 10),Horas frío (base 7)
0,06-01-2021 00:00,"17,4","0,0","66,5","1000,0","1,0","0,0","16,9","17,7","0,0","643,8",--
1,06-01-2021 01:00,"17,0","0,0","67,6","1000,0","1,0","0,0","15,9","17,5","0,0",--,--
2,06-01-2021 02:00,"15,5","0,0","71,9","1000,0","1,0","0,0","15,3","15,8","0,0",--,--
3,06-01-2021 03:00,"14,7","0,0","75,2","999,8","1,0","0,0","14,3","15,1","0,0",--,--
4,06-01-2021 04:00,"14,0","0,0","78,4","999,9","1,0","0,0","13,4","14,3","0,0",--,--


Let's build a dataframe with temperature and humidity data from sensors

In [6]:
sensor_th = data_sensor[data_sensor.measurement == 'air_temp'].append(data_sensor[data_sensor.measurement == 'air_hum'])

In [7]:
sensor_th_ok = sensor_th[sensor_th.sensor_status == 'FFC_SENSOR_OK']

In [8]:
sensor_th_ok.head()

,time,measurement,site_id,mote_id,sensor_id,sensor_status,sensor_value
213896,2020-12-03T00:00:02.233210,air_temp,parral,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,FFC_SENSOR_OK,19.279
213897,2020-12-03T00:00:02.233210,air_temp,parral,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x1,FFC_SENSOR_OK,19.957
213898,2020-12-03T00:00:02.233210,air_temp,parral,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x2,FFC_SENSOR_OK,20.341
213899,2020-12-03T00:00:02.233210,air_temp,parral,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x3,FFC_SENSOR_OK,20.654
213900,2020-12-03T00:00:03.477323,air_temp,parral,00-17-0d-00-00-59-a9-fb,FFC_SENSOR_SHT3x1,FFC_SENSOR_OK,19.936


Trying out different data manipulation techniques

In [52]:
#Pivot
sensor_th_ok['time'] = pd.to_datetime(sensor_th_ok['time'])
data_pivot = sensor_th_ok.pivot(index='time', columns=['mote_id', 'sensor_id', 'measurement'], values='sensor_value')

/var/folders/_z/x0zcrx6j7lqb_z4xl709hdqh0000gn/T/ipykernel_3854/2289852153.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_th_ok['time'] = pd.to_datetime(sensor_th_ok['time'])


In [53]:
data_pivot.head()

mote_id                    00-17-0d-00-00-59-cd-c9  ... 00-17-0d-00-00-59-ce-df
sensor_id                        FFC_SENSOR_SHT3x0  ...       FFC_SENSOR_SHT3x3
measurement                               air_temp  ...                 air_hum
time                                                ...                        
2020-12-03 00:00:02.233210                  19.279  ...                     NaN
2020-12-03 00:00:03.477323                     NaN  ...                     NaN
2020-12-03 00:00:04.178922                     NaN  ...                     NaN
2020-12-03 00:00:06.141080                     NaN  ...                     NaN
2020-12-03 00:00:07.407819                     NaN  ...                     NaN

[5 rows x 60 columns]

In [54]:
# Sampling to 10 minutes
data_pivot_resample = data_pivot.resample('10T').mean()

In [55]:
data_pivot_resample.head()

mote_id             00-17-0d-00-00-59-cd-c9  ... 00-17-0d-00-00-59-ce-df
sensor_id                 FFC_SENSOR_SHT3x0  ...       FFC_SENSOR_SHT3x3
measurement                        air_temp  ...                 air_hum
time                                         ...                        
2020-12-03 00:00:00               17.803617  ...               49.469915
2020-12-03 00:10:00               16.755356  ...               50.366367
2020-12-03 00:20:00               17.563883  ...               50.955508
2020-12-03 00:30:00               17.799083  ...               52.946817
2020-12-03 00:40:00               17.102186  ...               53.730610

[5 rows x 60 columns]

In [41]:
#GroupBy
data_group = sensor_th_ok.groupby(['time', 'mote_id', 'sensor_id', 'measurement'])['sensor_value'].agg('mean')

In [42]:
data_group.head(10)

time                        mote_id                  sensor_id          measurement
2020-12-03T00:00:02.233210  00-17-0d-00-00-59-cd-c9  FFC_SENSOR_SHT3x0  air_hum        58.409
                                                                        air_temp       19.279
                                                     FFC_SENSOR_SHT3x1  air_hum        55.331
                                                                        air_temp       19.957
                                                     FFC_SENSOR_SHT3x2  air_hum        51.982
                                                                        air_temp       20.341
                                                     FFC_SENSOR_SHT3x3  air_hum        50.712
                                                                        air_temp       20.654
2020-12-03T00:00:03.477323  00-17-0d-00-00-59-a9-fb  FFC_SENSOR_SHT3x1  air_hum        49.014
                                                                      

In [43]:
df_data_group = pd.DataFrame(data_group.unstack(level='measurement')).reset_index()

In [44]:
df_data_group.head()

measurement,time,mote_id,sensor_id,air_hum,air_temp
0,2020-12-03T00:00:02.233210,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,58.409,19.279
1,2020-12-03T00:00:02.233210,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x1,55.331,19.957
2,2020-12-03T00:00:02.233210,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x2,51.982,20.341
3,2020-12-03T00:00:02.233210,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x3,50.712,20.654
4,2020-12-03T00:00:03.477323,00-17-0d-00-00-59-a9-fb,FFC_SENSOR_SHT3x1,49.014,19.936


In [48]:
mote_names = df_data_group.mote_id.unique()
sensor_names = df_data_group.sensor_id.unique()

In [51]:
df_data_group[(df_data_group.mote_id=='00-17-0d-00-00-59-cd-c9') & (df_data_group.sensor_id=='FFC_SENSOR_SHT3x0')]

measurement,time,mote_id,sensor_id,air_hum,air_temp
0,2020-12-03T00:00:02.233210,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,58.409,19.279
33,2020-12-03T00:00:12.046546,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,64.761,19.265
63,2020-12-03T00:00:21.868299,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,66.592,19.223
90,2020-12-03T00:00:30.987425,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,68.008,19.151
120,2020-12-03T00:00:41.515315,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,68.821,19.009
...,...,...,...,...,...
253447,2020-12-03T23:59:15.319531,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,37.211,19.671
253477,2020-12-03T23:59:26.599519,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,37.170,19.615
253507,2020-12-03T23:59:35.565374,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,37.083,19.602
253537,2020-12-03T23:59:45.469356,00-17-0d-00-00-59-cd-c9,FFC_SENSOR_SHT3x0,37.373,19.559


Intento fallido

In [35]:
df_data_group['time'] = pd.to_datetime(df_data_group['time'])

In [37]:
df_data_group = df_data_group.set_index('time')

In [39]:
df_resample = df_data_group.resample('10T').mean()

In [40]:
df_resample.head()

measurement,air_hum,air_temp
time,,
2020-12-03 00:00:00,55.508740,19.014244
2020-12-03 00:10:00,57.482228,18.431225
2020-12-03 00:20:00,57.289368,18.376236
2020-12-03 00:30:00,58.816554,17.881276
2020-12-03 00:40:00,60.064678,17.390342
